# Exercises

## Thermodynamic states and simple processes

1. Calculate the density and enthalpy of R290 at 5 bar and 50 °C.
2. What is the enthalpy of saturated liquid Ammonia at 75 °C.
3. Calculate the vapor mass fraction in the two-phase region of Ammonia at the
   same enthalpy but at a temperature of 25 °C.
4. At what temperature does Ammonia start to boil under a pressure of 4 bar.
5. What pressures correspond to a saturation temperature of 20 °C, 65 °C and
   110 °C for water?
6. Calculate the entropy of R134a for the saturated vapor state at a temperature
   of 10 °C.
7. What is the temperature of R134a at the same entropy and twice as high
   pressure?


In [ ]:
from CoolProp.CoolProp import PropsSI

In [ ]:
PropsSI("D", "P", 5e5, "T", 50 + 273.15, "R290")

In [ ]:
h = PropsSI("H", "Q", 0, "T", 75 + 273.15, "ammonia")
h

In [ ]:
p = PropsSI("P", "T", 25 + 273.15, "Q", 0, "ammonia")
PropsSI("Q", "H", h, "P", p, "ammonia")

In [ ]:
PropsSI("T", "P", 4e5, "Q", 0, "ammonia") - 273.15

In [ ]:
PropsSI("P", "T", [t + 273.15 for t in [20, 65, 110]], "Q", 0, "water")

In [ ]:
s = PropsSI("S", "T", 10 + 273.15, "Q", 1, "R134a")
p = PropsSI("P", "T", 10 + 273.15, "Q", 1, "R134a")
s

In [ ]:
PropsSI("T", "S", s, "P", p * 2, "R134a") - 273.15


## Refrigerants

1. Plot the saturation curve for Ammonia, Water, R134a and Pentane in a log-p,T
   diagram for temperature ranging from -25 °C to 100 °C.
2. Compare the plots: What factors may restrict the usage of the working fluids
   in heat pumps?
3. Calculate the entropy for the dew line and the bubble line of the same
   working fluids for temperature values larger than -25 °C. Plot the lines in
   a T-s diagram.
4. Compare the plots: What factors may restrict the usage of the working fluids
   in heat pumps?
   

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
fluids = ["ammonia", "water", "R134a", "Pentane", "R290", "R600", "R1234ze(E)"]
temperature = np.linspace(-25, 125)
saturation_pressure = {}
for fluid in fluids:
    saturation_pressure[fluid] = PropsSI("P", "T", temperature + 273.15, "Q", 0, fluid)

fig, ax = plt.subplots(2, figsize=(12, 8))

for fluid in fluids:
    ax[0].plot(temperature, saturation_pressure[fluid], label=fluid)
    ax[1].plot(temperature, saturation_pressure[fluid] / saturation_pressure[fluid].min())

ax[0].set_yscale("log")
ax[0].set_ylabel("pressure in Pa")
ax[1].set_xlabel("temperature in °C")

ax[0].plot(temperature, [1e5] * len(temperature), "--", label="ambient pressure")
ax[0].legend()

_ = ax[1].set_ylabel("pressure ratio p / p$_\\text{min}$")

- Operation below ambient pressure is challenging, since a vacuum has to be
  preserved. Therefore, for low temperature applications water, pentane and R600
  are not very feasible.
- Similarly, very high pressure is requires a lot of technical effort. E.g. to
  reach 125 °C, Ammonia has to be compressed to about 100 bar.
- R290 and R134a have very good pressure ranges. R290 is explosive, R134a has a
  very high global warming potential (GWP) [https://ozone.unep.org/system/files/documents/RTOC-assessment%20-report-2022.pdf]
- Some reach critical point around 100 °C. In that region, the condensation
  section is small, therefore heat exchange has to happen with either larger
  surface area or higher temperature difference.


In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))

for fluid in fluids:
    temperature_range = np.linspace(-25 + 273.15, PropsSI("TCRIT", fluid))
    bubble_entropy = PropsSI("S", "T", temperature_range, "Q", 0, fluid)
    dew_entropy = PropsSI("S", "T", temperature_range, "Q", 1, fluid)
    _ = ax.plot(bubble_entropy, temperature_range - 273.15, label=fluid)
    color = _[0].get_color()
    ax.plot(dew_entropy, temperature_range - 273.15, color=color)

ax.set_ylabel("temperature in °C")
ax.set_xlabel("entropy in J/kgK")
_ = ax.legend()

- Dry fluids (dewline leans over isentropics) risk compression into two-phase
  region, which may cause damage to the compressor.
- Small two-phase region indicates low heat of condensation ($q=\int s dT$)
  compared to desuperheating, which means higher heat exchange surface area
  required.

## Component Models

### Compressor

Implement a model, that allows you to model a compressor, where the compression
process is adiabatic and reversible (isentropic).
   
1. What power does the compressor draw if 5 kg/s of R290 are compressed from
   saturated gaseous state at 15 °C to a pressure corresponding to a saturation
   temperature of 60 °C?
2. What is the outlet temperature?
3. How much mass can be compressed by the same machine, if 300 kW of power are
   available?

Now consider thermodynamic inefficiencies by incorporating isentropic
efficiency in the model.

4. Assume isentropic efficiency of 80 %. What is the power, what is the outlet
   temperature?
5. The outlet temperature is measured to be 75 °C. What is the isentropic
   efficiency?
6. How does the compressor power requirement change as function of the
   isentropic efficiency with the starting point of the compression as in the
   first task. Plot the results.


In [ ]:
fluid = "R290"
m = 5
t_1 = 15 + 273.15
t_2sat = 60 + 273.15

p_1 = PropsSI("P", "T", t_1, "Q", 1, fluid)

h_1 = PropsSI("H", "P", p_1, "Q", 1, fluid)
s_1 = PropsSI("S", "P", p_1, "Q", 1, fluid)

p_2 = PropsSI("P", "T", t_2sat, "Q", 1, fluid)
h_2s = PropsSI("H", "S", s_1, "P", p_2, fluid)

power = m * (h_2s - h_1)
power

In [ ]:
t_2s = PropsSI("T", "H", h_2s, "P", p_2, fluid)
t_2s - 273.15

In [ ]:
power = 1e6
m = power / (h_2s - h_1)
m

In [ ]:
eta_s = 0.8
m = 5
h_2 = h_1 + (h_2s - h_1) / eta_s
power = m * (h_2 - h_1)
power

In [ ]:
PropsSI("T", "H", h_2, "P", p_2, fluid) - 273.15

In [ ]:
t_2 = 75
h_2 = PropsSI("H", "T", t_2 + 273.14, "P", p_2, fluid)
(h_2s - h_1) / (h_2 - h_1)

In [ ]:
import numpy as np

eta_s_range = np.linspace(0.6, 0.9)
h_2_range = h_1 + (h_2s - h_1) / eta_s_range
t_2_range = PropsSI("T", "P", p_2, "H", h_2_range, fluid)
power_range = m * (h_2_range - h_1)

fig, ax = plt.subplots(2, figsize=(8, 4.5))

eta_s_range *= 100
power_range /= 1e3
t_2_range -= 273.15

ax[0].plot(eta_s_range, power_range)
ax[0].set_ylabel("power input in kW")
ax[1].plot(eta_s_range, t_2_range)
ax[1].set_ylabel("outlet temperature in °C")
ax[1].set_xlabel("isentropic efficiency in %")

ax[0].grid()
ax[1].grid()


### Heat exchanger

1. How much heat is required to completely evaporate 3 kg/s saturated liquid
   R134a at 10, 20 and 30 °C?
2. Which volumetric flow of air is required to provide the heat in any of the 
   three cases? Assume, that the minimal temperature difference between the air
   and the working fluid is 3 °C and the air temperature changes by 5 °C.
3. Which volumetric flow of water is required under the same circumstances?
4. What information can you transfer to the heat exchanger design?

In [ ]:
m = 3
working_fluid = "R134a"
t = [_t + 273.15 for _t in [10, 20, 30]]
h_1 = PropsSI("H", "T", t, "Q", 0, working_fluid)
h_2 = PropsSI("H", "T", t, "Q", 1, working_fluid)

heat = m * (h_2 - h_1)
heat

In [ ]:
t_source_2 = np.array(t) + 3
t_source_1 = t_source_2 + 5
p_source = 1e5
fluid = "air"

h_source_1 = PropsSI("H", "T", t_source_1, "P", p_source, fluid)
h_source_2 = PropsSI("H", "T", t_source_2, "P", p_source, fluid)

v_source_1 = 1 / PropsSI("D", "T", t_source_1, "P", p_source, fluid)
m_source = -heat / (h_source_2 - h_source_1)
m_source

In [ ]:
m_source * v_source_1

In [ ]:
fluid = "water"

h_source_1 = PropsSI("H", "T", t_source_1, "P", p_source, fluid)
h_source_2 = PropsSI("H", "T", t_source_2, "P", p_source, fluid)

v_source_1 = 1 / PropsSI("D", "T", t_source_1, "P", p_source, fluid)
m_source = -heat / (h_source_2 - h_source_1)
m_source

In [ ]:
m_source * v_source_1

- High volumetric flow means either high flow speed or very large heat exchange
  area.
- Pressure losses are extra costly with high volumetric flow, blowers/fans draw
  a lot of power.
- Water has higher density and also higher mass specific heat capacity. Change
  of temperature is lower for any given flowrate.
